<a href="https://colab.research.google.com/github/amrutask/Machine_Learning/blob/master/HomeWork2/CNN_arc3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [13]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print("X_train Size :",X_train.shape)
print("X_test Size : ",X_test.shape)
print(y_train.shape)

X_train Size : (50000, 32, 32, 3)
X_test Size :  (10000, 32, 32, 3)
(50000, 1)


In [14]:
train_X, val_X, train_Y, val_Y = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print("Size of training data :",train_X.shape)
print("Size of validation data : ",val_X.shape)

Size of training data : (40000, 32, 32, 3)
Size of validation data :  (10000, 32, 32, 3)


In [15]:
num_classes=10
epoch=100

train_Y = keras.utils.to_categorical(train_Y, num_classes)
val_Y = keras.utils.to_categorical(val_Y, num_classes)
test_Y= keras.utils.to_categorical(y_test, num_classes)

print("Size of train, validation and test labels: ", train_Y.shape, val_Y.shape, test_Y.shape) 

Size of train, validation and test labels:  (40000, 10) (10000, 10) (10000, 10)


In [16]:
model = Sequential()

model.add(Conv2D(32, (3, 3), strides=(1, 1), input_shape=train_X.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(112, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(112, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(700, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 112)       32368     
_________________________________________________________________
activation_8 (Activation)    (None, 15, 15, 112)       0         
__________

In [0]:
train_X = train_X.astype('float32')
val_X = val_X.astype('float32')
X_test = X_test.astype('float32')
train_X /= 255
X_test /= 255
val_X /= 255

In [18]:
datagen = ImageDataGenerator(featurewise_center=False,samplewise_center=False,featurewise_std_normalization=False,samplewise_std_normalization=False, zca_whitening=False,
        zca_epsilon=1e-06, rotation_range=0,width_shift_range=0.1, height_shift_range=0.1, shear_range=0., zoom_range=0.,channel_shift_range=0., fill_mode='nearest',
        cval=0.,
        horizontal_flip=False,
        vertical_flip=True,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.1)

datagen.fit(train_X)
model.fit_generator(datagen.flow(train_X, train_Y), steps_per_epoch=1000, epochs=epoch, validation_data=(val_X, val_Y))

Epoch 1/100
1000/1000 [==============================] - 57s 57ms/step - loss: 1.8168 - acc: 0.3420 - val_loss: 1.7633 - val_acc: 0.3776
Epoch 2/100
1000/1000 [==============================] - 53s 53ms/step - loss: 1.5446 - acc: 0.4375 - val_loss: 1.7628 - val_acc: 0.4139
Epoch 3/100
1000/1000 [==============================] - 54s 54ms/step - loss: 1.4224 - acc: 0.4859 - val_loss: 1.9600 - val_acc: 0.4070
Epoch 4/100
1000/1000 [==============================] - 52s 52ms/step - loss: 1.3599 - acc: 0.5104 - val_loss: 1.7786 - val_acc: 0.4617
Epoch 5/100
1000/1000 [==============================] - 52s 52ms/step - loss: 1.2982 - acc: 0.5336 - val_loss: 1.6686 - val_acc: 0.4899
Epoch 6/100
1000/1000 [==============================] - 52s 52ms/step - loss: 1.2446 - acc: 0.5543 - val_loss: 2.4492 - val_acc: 0.3898
Epoch 7/100
1000/1000 [==============================] - 53s 53ms/step - loss: 1.2108 - acc: 0.5674 - val_loss: 2.4763 - val_acc: 0.3895
Epoch 8/100
1000/1000 [==================

In [19]:
score = model.evaluate(val_X, val_Y, verbose=0) 
print('Validation score:', score[0]) 
print('Validation accuracy:', score[1])

Validation score: 0.9125799599170685
Validation accuracy: 0.7574
